In [6]:
import pandas as pd
import numpy as np
import torch
import os
from transformers import AutoTokenizer, AutoModel

!pip install openpyxl

!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 2.1 MB/s eta 0:00:00


In [7]:
#Connecting to the drive
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [8]:
# Code to change the directory path where the input file is present
os.chdir('/content/drive/MyDrive/')


In [9]:
# Load the Excel file
file_path = 'vecsBERTinput.xlsx'##change filename to your filename
sheet1 = 'positive triplets'
sheet2 = 'negative triplets'
sheet3 = 'vecs for MWTs'

In [10]:
#Testing for commit
# Read the Excel file into a pandas DataFrame
df_sheet1 = pd.read_excel(io=file_path, sheet_name=sheet1)
df_sheet1.columns = df_sheet1.columns.str.strip()
df_sheet1 = df_sheet1.dropna(subset=['Hypernym'])



In [11]:
#find related merged terms sheet1

hypernym = df_sheet1['Hypernym']
concated_list = []
concated_list_values = ''
maxlen = len(df_sheet1['Hypernym'])
related_index = 0

# Group the DataFrame by 'Hypernym' and aggregate 'Anchor term' into a list
anchor_terms_by_hypernym = df_sheet1.groupby('Hypernym')['Anchor term'].apply(list).reset_index()


for index, row in df_sheet1.iterrows():
    closely_related_term = row['Closely related term']
    distantly_related_term = row['Distantly related term']
    hypernym_exclude = row['Hypernym']

    # Retrieve anchor terms based on 'Hypernym'
    if hypernym_exclude in anchor_terms_by_hypernym['Hypernym'].values:
        anchor_terms_exclude = anchor_terms_by_hypernym[anchor_terms_by_hypernym['Hypernym'] == hypernym_exclude]['Anchor term'].iloc[0]
    else:
        anchor_terms_exclude = []

    if closely_related_term in anchor_terms_exclude or distantly_related_term in anchor_terms_exclude:
        if closely_related_term not in anchor_terms_exclude:
            concat_crt_drt = f"{closely_related_term}"
        if distantly_related_term not in anchor_terms_exclude:
            concat_crt_drt = f"{distantly_related_term}"
    else:
        concat_crt_drt = f"{closely_related_term}, {distantly_related_term}"

    if concated_list_values == '':
        concated_list_values = concat_crt_drt
    else:
        concated_list_values = f"{concated_list_values}, {concat_crt_drt}"

    related_index += 1
    if index < maxlen - 1:
        if hypernym[index] != hypernym[index+1]:

            for ri in range(related_index):
                concated_list.append(concated_list_values)

            concated_list_values = ''
            related_index = 0
    else:
        for ri in range(related_index):
            concated_list.append(concated_list_values)

df_sheet1['Related terms merged'] = concated_list

In [12]:
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

# negative triplets sheet2

df_sheet2 = pd.read_excel(io=file_path, sheet_name=sheet2)

anchor_terms = df_sheet1['Anchor term'].tolist()
related_terms = df_sheet1['Related terms merged'].tolist()
closely_terms = df_sheet1['Closely related term'].tolist()
distantly_terms = df_sheet1['Distantly related term'].tolist()

# Create a list to store the duplicated anchor terms
anchor_dup = []
new_crt_1 = []
new_drt_1 = []
new_crt_2 = []
new_drt_2 = []
#maxlen =len(related_terms)
# Create new rows for each related term
for i, terms in enumerate(related_terms):

    for term in terms.split(', '):

            anchor_dup.append(anchor_terms[i])
            new_crt_1.append(term)
            new_drt_1.append(closely_terms[i])
            new_crt_2.append(distantly_terms[i])
            new_drt_2.append(term)


data_combined_1 = {'Anchor term': anchor_dup, 'Closely related term': new_crt_1,'Distantly related term':new_drt_1}
df_combined1 = pd.DataFrame(data_combined_1)

data_combined_2 = {'Anchor term': anchor_dup, 'Closely related term': new_crt_2,'Distantly related term':new_drt_2}
df_combined2 = pd.DataFrame(data_combined_2)

# result = df_combined1.append(df_combined2, ignore_index=True)
result = pd.concat([df_combined1,df_combined2])

df_sheet2 = result[result['Closely related term'] != result['Distantly related term']]

df_sheet2.sort_values(by=['Anchor term', 'Closely related term', 'Distantly related term'], inplace=True)
df_sheet2.drop_duplicates(inplace=True)

<ipython-input-12-cb365df53182>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet2.sort_values(by=['Anchor term', 'Closely related term', 'Distantly related term'], inplace=True)
<ipython-input-12-cb365df53182>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sheet2.drop_duplicates(inplace=True)


In [13]:
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Load a model and tokenizer
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

#read the sheet3
df_sheet3 = pd.read_excel(io=file_path, sheet_name=sheet3)
MWT = df_sheet3['MWT']

df_sheet3['MEAN-pooling (M_W_T)'] =MWT.apply(lambda x:
    torch.mean(model(**tokenizer(x, return_tensors='pt', padding=True, truncation=True)).last_hidden_state, dim=1).squeeze().detach().numpy().tolist())

df_sheet3['MAX-pooling (M_W_T)'] = MWT.apply(lambda x:
    torch.max(model(**tokenizer(x, return_tensors='pt', padding=True, truncation=True)).last_hidden_state, dim=1).values.squeeze().detach().numpy().tolist())

df_sheet3['CLS-token pooling (M_W_T)'] = MWT.apply(lambda x:
    model(**tokenizer(x, return_tensors='pt', padding=True, truncation=True)).last_hidden_state[:, 0, :].squeeze().detach().numpy().tolist())

df_sheet3['MIN-pooling (M_W_T)'] = MWT.apply(lambda x:
    torch.min(model(**tokenizer(x, return_tensors='pt', padding=True, truncation=True)).last_hidden_state, dim=1).values.squeeze().detach().numpy().tolist())



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
#combine in one excel file
output_excel = 'vecsScivocUNCoutput.xlsx'

# Create an ExcelWriter object
with pd.ExcelWriter(output_excel) as writer:
    # Write the new DataFrame to a sheet
    df_sheet1.to_excel(writer, sheet_name='positive triplets', index=False)
    df_sheet2.to_excel(writer, sheet_name='negative triplets', index=False)
    df_sheet3.to_excel(writer, sheet_name='vecs for MWTs', index=False)

**TASK 2**

In [15]:
"""
Task 2.1. Reconstruction of triplets
Input: A set of positive MWT triplets + A list of vector representation of MWTs
Output: Pooling method corresponding lists of MWT triplets
"""
# import libraries

# import numpy as np
# import pandas as pd
import openpyxl

#Specify the output filename of task 1
input_file_name = 'vecsScivocUNCoutput.xlsx' ##################Please change respected input file name

#Specify the input filename for task 2.1
input_filename = 'evalBERTinput.xlsx'

#Specify the output filename for task 2.1
output_filename = 'evalScivocUNCoutput.xlsm'

# function to calculate cosine similarity between two vectors
def cosine_sim(vec1, vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

# name of pooling method used in Experiment2 task 1.2 output
# pooling_methods = ["MEAN-pooling", "MAX-pooling", "CLS-token pooling", "MIN-pooling"]
pooling_methods = ["MEAN-pooling (M_W_T)", "MAX-pooling (M_W_T)", "CLS-token pooling (M_W_T)", "MIN-pooling (M_W_T)"]

column_index = 1 # use to chnage columns for evaluation output
output_format_file = openpyxl.load_workbook(input_filename, read_only=False, keep_vba=True)
sheetname = output_format_file.get_sheet_by_name('evaluation') #get evaluation sheet to store evaluation outputs
positive_triplets_df = pd.read_excel(input_file_name, "positive triplets") # read positive triplets
negative_triplets_df = pd.read_excel(input_file_name, "negative triplets") # read negative triplets
triplet_sheetname_count = 1
task1_2_df = pd.read_excel(input_file_name, "vecs for MWTs")

for pooling_methid_index, pooling_method in enumerate(pooling_methods):
    triplet_sheetname_count = triplet_sheetname_count + 1
    triplets_sheet = output_format_file.get_sheet_by_name(f'triplets{triplet_sheetname_count}')

    # dataframe which contains triplets which match the condition
    pooling_method_op = pd.DataFrame(columns=["Anchor term", "Closely related term", "Distantly related term"])
    row_start_index = 3 # used to arrange the output in specific cell
    column_range = [1,2,3]

    # Combine the anchor term, closely related term and distantly related terms who has same Related terms merged and calculate cosine similarity between each anchor term with closely related terms and distantly related term
    for inn, related_word in enumerate(positive_triplets_df["Related terms merged"].unique()):
        # dataframe that has same "Related terms merged"
        df_with_same_related_word = positive_triplets_df.loc[positive_triplets_df["Related terms merged"] == related_word]
        for i, a_term in enumerate(df_with_same_related_word["Anchor term"]):
                # match anchor term with MWT in task 1.2 and get the respected pooling method vector and convert from string to list
                anchor_term_vector = task1_2_df.loc[task1_2_df['MWT'] == a_term][pooling_method].values[0].replace('[',"").replace("]","").replace(" ","").split(",")
                anchor_term_vector = [float(val) for val in anchor_term_vector]

                for row_index, row in df_with_same_related_word.iterrows():
                    # Same as anchor term match "Closely related term" and "Distantly related term" with MWT and get respected pooling method vector
                    c_term = row["Closely related term"]
                    d_term = row["Distantly related term"]
                    # If the "Closely related term" or "Distantly related term" not match with any MWT in task 1.2 then skip that term
                    if len(task1_2_df.loc[task1_2_df['MWT'] == c_term][pooling_method].values) < 1 or len(task1_2_df.loc[task1_2_df['MWT'] == d_term][pooling_method].values) < 1:
                        continue
                    closely_releted_term_vector = task1_2_df.loc[task1_2_df['MWT'] == c_term][pooling_method].values[0].replace('[',"").replace("]","").replace(" ","").split(",")
                    closely_releted_term_vector = [float(val) for val in closely_releted_term_vector if val]

                    distantly_releted_term_vector = task1_2_df.loc[task1_2_df['MWT'] == d_term][pooling_method].values[0].replace('[',"").replace("]","").replace(" ","").split(",")
                    distantly_releted_term_vector = [float(val) for val in distantly_releted_term_vector if val]

                    # Calculate the cosine similarity between "Anchor term" and "Closely related term"
                    cossim_anch_clos = cosine_sim(anchor_term_vector, closely_releted_term_vector)
                    # Calculate the cosine similarity between "Anchor term" and "Distantly related term"
                    cossim_anch_dist = cosine_sim(anchor_term_vector, distantly_releted_term_vector)
                    # If condition match the add triplet in dataframe and in a specific cell of the file
                    if(cossim_anch_clos > cossim_anch_dist):
                        triplets_sheet.cell(row=row_start_index+1, column=column_range[0]).value = a_term
                        triplets_sheet.cell(row=row_start_index+1, column=column_range[1]).value = c_term
                        triplets_sheet.cell(row=row_start_index+1, column=column_range[2]).value = d_term
                        row_start_index=row_start_index+1
                        pooling_method_op.loc[len(pooling_method_op.index)-1] = [a_term, c_term, d_term]

    # TASK 2.2 started, Here we evaluate pooling_method_op dataframe by calculating TP, TN, FP,FN, Precision, Recall, F1-score and Accuracy
    row_range = range(2,10)

    cols = ['Anchor term', 'Closely related term', 'Distantly related term']
    # get the rows that are not matched with positive triplets
    fn = len(positive_triplets_df.merge(pooling_method_op[cols], on=cols, how='outer', indicator=True).loc[lambda x: x.pop('_merge').eq('left_only')])
    tp = len(positive_triplets_df) - fn

    # get the rows that are not matched with negative triplets
    tn = len(negative_triplets_df.merge(pooling_method_op[cols], on=cols, how='outer', indicator=True).loc[lambda x: x.pop('_merge').eq('left_only')])
    fp = len(negative_triplets_df) - tn

    precision = (tp / (tp+fp)) if (tp+fp) > 0 else 0
    recall = (tp / (tp+fn)) if (tp+fn) > 0 else 0
    f1_score = 2 * ((precision * recall) / (precision + recall)) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) > 0 else 0
    dataToStore = [tp, fp, tn, fn, precision, recall, f1_score, accuracy]
    dataToStoreInitIndex = 0
    for row_index in row_range:
        sheetname.cell(row=row_index, column=column_index+1).value = dataToStore[dataToStoreInitIndex]
        dataToStoreInitIndex = dataToStoreInitIndex + 1
    column_index = column_index + 1
output_format_file.save(output_filename)


<ipython-input-15-b70e9b998a16>:31: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  sheetname = output_format_file.get_sheet_by_name('evaluation') #get evaluation sheet to store evaluation outputs
<ipython-input-15-b70e9b998a16>:39: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  triplets_sheet = output_format_file.get_sheet_by_name(f'triplets{triplet_sheetname_count}')
<ipython-input-15-b70e9b998a16>:39: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  triplets_sheet = output_format_file.get_sheet_by_name(f'triplets{triplet_sheetname_count}')
<ipython-input-15-b70e9b998a16>:39: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  triplets_sheet = output_format_file.get_sheet_by_name(f'triplets{triplet_sheetname_count}')
<ipython-input-15-b70e9b998a16>:39: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[s